In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Cristóvão Beirão da Cruz e Silva <cbeiraod@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

#############################################################################
# Modified for ETROC2 I2C testing in jupyter notebooks, Murtaza Safdari
#############################################################################

## Imports

In [ ]:
from glob import glob
from natsort import natsorted
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import mplhep as hep
hep.style.use('CMS')
from pathlib import Path
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os, sys
import datetime
from fnmatch import fnmatch
import scipy.stats as stats
from math import ceil
from numpy import savetxt
import pickle

In [ ]:
def multiple_trigger_bit_noisescan_plot(root, file_pattern, chip_figtitle, chip_figname, scan_list, attempts=[], tags=[], colors = ['k']):
    scan_name = chip_figname+"_VRef_SCurve_NoiseOnly"
    triggerbit_full_Scurve = {row:{col:{attempt:{} for attempt in attempts} for col in range(16)} for row in range(16)}

    today = datetime.date.today().isoformat()

    fig_outdir = Path('../../ETROC-figures')
    fig_outdir = fig_outdir / (today + '_Array_Test_Results')
    fig_outdir.mkdir(exist_ok=True)
    fig_path = str(fig_outdir)

    row_list, col_list = zip(*scan_list)
    u_cl = np.sort(np.unique(col_list))
    u_rl = np.sort(np.unique(row_list))
    
    for row,col in scan_list:
        for attempt in attempts:
            path_pattern = f"*_Array_Test_Results*/{scan_name}_Pixel_C{col}_R{row}"+attempt
            file_list = []
            for path, subdirs, files in os.walk(root):
                if not fnmatch(path, path_pattern): continue
                for name in files:
                    pass
                    if fnmatch(name, file_pattern):
                        file_list.append(os.path.join(path, name))
            # print(file_list)
            for file_index, file_name in enumerate(file_list):
                with open(file_name) as infile:
                    for line in infile:
                        text_list = line.split(',')
                        FPGA_triggerbit = int(text_list[5])
                        DAC = int(text_list[-1])
                        if DAC == -1: continue
                        triggerbit_full_Scurve[row][col][attempt][DAC] = FPGA_triggerbit

    fig = plt.figure(dpi=200, figsize=(len(np.unique(u_cl))*16,len(np.unique(u_rl))*10))
    gs = fig.add_gridspec(len(np.unique(u_rl)),len(np.unique(u_cl)))
    for ri,row in enumerate(u_rl):
        for ci,col in enumerate(u_cl):
            x_max = 0
            x_min = 1000
            ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
            for attempt, tag, color in zip(attempts, tags, colors):
                Y = np.array(list(triggerbit_full_Scurve[row][col][attempt].values()))
                X = np.array(list(triggerbit_full_Scurve[row][col][attempt].keys()))
                ax0.plot(X, Y, '.-', color=color,lw=1.0,label=tag)
                ax0.set_xlabel("DAC Value [decimal]")
                ax0.set_ylabel("Trigger Bit Counts [decimal]")
                hep.cms.text(loc=0, ax=ax0, text="Preliminary", fontsize=25)
                min_x_point = X[np.argmin(Y)]
                max_x_point = X[np.argmax(Y)]
                if(max_x_point > x_max):
                    x_max = max_x_point
                if(min_x_point < x_min):
                    x_min = min_x_point
            ax0.set_xlim(left=x_min-x_min*0.02, right=x_max+x_max*0.02)
            plt.yscale("log")
            plt.title(f"{chip_figtitle}, Pixel ({row},{col}) Noise Peak"+tag,size=25, loc="right")
            plt.tight_layout()
            plt.legend(loc="upper right", fontsize=14)
    plt.savefig(fig_path+"/"+chip_figname+"_NoisePeak_Log"+attempts[0]+"_multiple_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    plt.close()

    fig = plt.figure(dpi=200, figsize=(len(np.unique(u_cl))*16,len(np.unique(u_rl))*10))
    gs = fig.add_gridspec(len(np.unique(u_rl)),len(np.unique(u_cl)))
    for ri,row in enumerate(u_rl):
        for ci,col in enumerate(u_cl):
            x_max = 0
            x_min = 1000
            ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
            for attempt, tag, color in zip(attempts, tags, colors):
                Y = np.array(list(triggerbit_full_Scurve[row][col][attempt].values()))
                X = np.array(list(triggerbit_full_Scurve[row][col][attempt].keys()))
                ax0.plot(X, Y, '.-', color=color,lw=1.0,label=tag)
                ax0.set_xlabel("DAC Value [decimal]")
                ax0.set_ylabel("Trigger Bit Counts [decimal]")
                hep.cms.text(loc=1, ax=ax0, text="Preliminary", fontsize=25)
                min_x_point = X[np.argmin(Y)]
                max_x_point = X[np.argmax(Y)]
                if(max_x_point > x_max):
                    x_max = max_x_point
                if(min_x_point < x_min):
                    x_min = min_x_point
            ax0.set_xlim(left=x_min-x_min*0.02, right=x_max+x_max*0.02)
            plt.yscale("linear")
            plt.title(f"{chip_figtitle}, Pixel ({row},{col}) Noise Peak"+tag,size=25, loc="right")
            plt.tight_layout()
            plt.legend(loc="upper right", fontsize=14)
    plt.savefig(fig_path+"/"+chip_figname+"_NoisePeak_Linear"+attempts[0]+"_multiple_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    plt.close()
    del triggerbit_full_Scurve

In [ ]:
root = '/home/daq/ETROC2/ETROC-Data/cern_Jan2024/TID/'
file_pattern = "*FPGA_Data.dat"
chip_figtitle = "ET2p01_W36_6"
chip_figname = "ET2p01_W36_6"
row_list = [0, 0, 3, 3, 12, 12]
col_list = [14, 2, 14, 2, 14, 2]
scan_list = list(zip(row_list, col_list))
attempts = ["_m25c_0MRad","_m25c_200MRad","_m25c_400MRad"]
tags = ["12C 0 MRad","-25C 200 MRad","-25C 400 MRad"]
colors = ['k','r','b']
multiple_trigger_bit_noisescan_plot(root,file_pattern,chip_figtitle,chip_figname,scan_list,attempts,tags,colors)

## QInj vs DAC

In [ ]:
def load_auto_cal_BL_map(chip_name, user_path="", day=datetime.date.today().isoformat()):
    indir = Path('../../ETROC-Data/'+(day+'_Array_Test_Results/')+user_path)
    infile_BL_map = indir / (chip_name+"_BL_map.pickle")
    with open(infile_BL_map, 'rb') as f:
        return pickle.load(f)

def load_auto_cal_NW_map(chip_name, user_path="", day=datetime.date.today().isoformat()):
    indir = Path('../../ETROC-Data/'+(day+'_Array_Test_Results/')+user_path)
    infile_NW_map = indir / (chip_name+"_NW_map.pickle")
    with open(infile_NW_map, 'rb') as f:
        return pickle.load(f)

In [ ]:
def charge_peakDAC_plot(chip_name, chip_figtitle, chip_figname, scan_list, QInjEns, attempt='', tag='', day = ''):
    scan_name = chip_figname+"_VRef_SCurve_BinarySearch_TurnOff"
    BL_map_THCal=load_auto_cal_BL_map(chip_name, "", day)
    NW_map_THCal=load_auto_cal_NW_map(chip_name, "", day)
    QInj_Peak_DAC_map = {row:{col:{q:0 for q in QInjEns} for col in range(16)} for row in range(16)}

    today = datetime.date.today().isoformat()

    fig_outdir = Path('../../ETROC-figures')
    fig_outdir = fig_outdir / (today + '_Array_Test_Results')
    fig_outdir.mkdir(exist_ok=True)
    fig_path = str(fig_outdir)

    for row,col in scan_list:
        for QInj in QInjEns:
            threshold_name = scan_name+f'_Pixel_C{col}_R{row}_QInj_{QInj}'+attempt
            path_pattern = f"*{today}_Array_Test_Results/{threshold_name}"
            file_list = []
            for path, subdirs, files in os.walk(root):
                if not fnmatch(path, path_pattern): continue
                for name in files:
                    pass
                    if fnmatch(name, file_pattern):
                        file_list.append(os.path.join(path, name))
            for file_index, file_name in enumerate(file_list):
                with open(file_name) as infile:
                    last_line = infile.readlines()[-1]
                    text_list = last_line.split(',')
                    DAC = int(text_list[-1])
                    QInj_Peak_DAC_map[row][col][QInj] = DAC

    row_list, col_list = zip(*scan_list)
    u_cl = np.sort(np.unique(col_list))
    u_rl = np.sort(np.unique(row_list))
    fig = plt.figure(dpi=200, figsize=(len(np.unique(u_cl))*16,len(np.unique(u_rl))*10))
    gs = fig.add_gridspec(len(np.unique(u_rl)),len(np.unique(u_cl)))
    for ri,row in enumerate(u_rl):
        for ci,col in enumerate(u_cl):
            BL = int(np.floor(BL_map_THCal[row][col]))
            NW = abs(int(np.floor(NW_map_THCal[row][col])))
            ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
            ax0.axhline(BL, color='k', lw=0.8, label=f"BL = {BL} DAC LSB")
            ax0.axhline(BL+NW, color='k',ls="--", lw=0.8, label=f"NW = $\pm${NW} DAC LSB")
            ax0.axhline(BL-NW, color='k',ls="--", lw=0.8)
            X = []
            Y = []
            for QInj in QInjEns:
                ax0.plot(QInj, QInj_Peak_DAC_map[row][col][QInj], 'rx')
                X.append(QInj)
                Y.append(QInj_Peak_DAC_map[row][col][QInj])
            X = np.array(X[:])
            Y = np.array(Y[:])
            (m, b), cov = np.polyfit(X, Y, 1, cov = True)
            n = Y.size
            Yfit = np.polyval((m,b), X)
            errorbars = np.sqrt(np.diag(cov))
            x_range = np.linspace(0, 35, 100)
            y_est = b + m*x_range
            resid = Y - Yfit
            s_err = np.sqrt(np.sum(resid**2)/(n - 2))
            t = stats.t.ppf(0.95, n - 2)
            ci2= t * s_err * np.sqrt(    1/n + (x_range - np.mean(X))**2/(np.sum((X)**2)-n*np.sum((np.mean(X))**2)))

            ax0.plot(x_range, y_est, 'b-', lw=-.8, label=f"DAC_TH = ({m:.2f}$\pm${errorbars[0]:.2f})$\cdot$Q + ({b:.2f}$\pm${errorbars[1]:.2f})")
            plt.fill_between(x_range, y_est+ci2, y_est-ci2, color='b',alpha=0.2, label="95% Confidence Interval on Linear Fit")
            ax0.set_xlabel("Charge Injected [fC]")
            ax0.set_ylabel("DAC Threshold [LSB]")
            hep.cms.text(loc=0, ax=ax0, text="Preliminary", fontsize=25)
            plt.title(f"{chip_figtitle}, Pixel ({row},{col}) Qinj Sensitivity Plot"+tag, size=25, loc='right')
            plt.legend(loc=(0.04,0.65))
            plt.tight_layout()
    plt.savefig(fig_path+"/"+chip_figname+"_QInj_Sensitivity"+attempt+"_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    plt.close()
    del QInj_Peak_DAC_map

In [ ]:
charge_peakDAC_plot(chip_name="ET2p01_W36_6", chip_figtitle="ET2p01_W36_6", chip_figname="ET2p01_W36_6", scan_list=scan_list, QInjEns=[5, 6, 8, 15, 22, 27], attempt='p12c_0MRad', tag='', day = '2024-01-10')